In [5]:

import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader

In [2]:
dataset = Planetoid(root='/tmp/PubMed', name='PubMed',transform=T.ToSparseTensor())
data = dataset[0]
#data.adj_t = data.adj_t.to_symmetric()
#data.adj_t = data.adj_t.to_symmetric()
print(data)
#split_idx = dataset.get_idx_split()
#train_idx = split_idx['train'].to(device)

Data(x=[19717, 500], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], adj_t=[19717, 19717, nnz=88648])


In [3]:
train_index = np.where(data.train_mask)[0]
print(len(train_index))
valid_index = np.where(data.val_mask)[0]
print(len(valid_index))
test_index = np.where(data.test_mask)[0]
print(len(test_index))

60
500
1000


# GCN using only domain Feature

In [3]:
import torch


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [4]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    #print(len(out))
    #print(data.y.squeeze(1)[train_idx])
    loss = F.nll_loss(out, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total

@torch.no_grad()
def test(model, data, train_idx,valid_idx,test_idx):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [6]:
pred=torch.tensor([1, 0, 1,0])
label=torch.tensor([1, 1, 1,0])
correct = pred.eq(label).sum().item()
total=len(label)
print(correct)
print(total)
print(correct/total)
print(ACC(pred,label))

3
4
0.75
0.75


In [7]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    dataset = Planetoid(root='/tmp/PubMed', name='PubMed',transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GCN(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.3147, Train: 51.67%, Valid: 45.60% Test: 45.10%
Run: 01, Epoch: 02, Loss: 0.7337, Train: 50.00%, Valid: 43.40% Test: 43.50%
Run: 01, Epoch: 03, Loss: 0.4566, Train: 51.67%, Valid: 42.80% Test: 43.90%
Run: 01, Epoch: 04, Loss: 0.3187, Train: 50.00%, Valid: 42.60% Test: 44.00%
Run: 01, Epoch: 05, Loss: 0.2322, Train: 50.00%, Valid: 42.40% Test: 44.30%
Run: 01, Epoch: 06, Loss: 0.2217, Train: 53.33%, Valid: 44.80% Test: 46.00%
Run: 01, Epoch: 07, Loss: 0.1565, Train: 58.33%, Valid: 47.00% Test: 47.70%
Run: 01, Epoch: 08, Loss: 0.1183, Train: 61.67%, Valid: 49.00% Test: 50.50%
Run: 01, Epoch: 09, Loss: 0.1641, Train: 78.33%, Valid: 52.20% Test: 54.00%
Run: 01, Epoch: 10, Loss: 0.0993, Train: 81.67%, Valid: 55.80% Test: 56.40%
Run: 01, Epoch: 11, Loss: 0.0725, Train: 88.33%, Valid: 59.80% Test: 59.80%
Run: 01, Epoch: 12, Loss: 0.0812, Train: 95.00%, Valid: 62.00% Test: 62.40%
Run: 01, Epoch: 13, Loss: 0.0817, Train: 96.67%, Valid: 63.00% Test: 64.20%
Run: 01, Epo

Run: 01, Epoch: 112, Loss: 0.0018, Train: 100.00%, Valid: 71.80% Test: 72.20%
Run: 01, Epoch: 113, Loss: 0.0002, Train: 100.00%, Valid: 71.80% Test: 72.20%
Run: 01, Epoch: 114, Loss: 0.0126, Train: 100.00%, Valid: 71.80% Test: 72.20%
Run: 01, Epoch: 115, Loss: 0.0043, Train: 100.00%, Valid: 71.80% Test: 72.20%
Run: 01, Epoch: 116, Loss: 0.0095, Train: 100.00%, Valid: 72.00% Test: 72.10%
Run: 01, Epoch: 117, Loss: 0.0015, Train: 100.00%, Valid: 72.00% Test: 72.30%
Run: 01, Epoch: 118, Loss: 0.0037, Train: 100.00%, Valid: 72.00% Test: 72.50%
Run: 01, Epoch: 119, Loss: 0.0018, Train: 100.00%, Valid: 72.00% Test: 72.70%
Run: 01, Epoch: 120, Loss: 0.0029, Train: 100.00%, Valid: 72.00% Test: 72.70%
Run: 01, Epoch: 121, Loss: 0.0012, Train: 100.00%, Valid: 72.00% Test: 72.50%
Run: 01, Epoch: 122, Loss: 0.0040, Train: 100.00%, Valid: 72.20% Test: 72.80%
Run: 01, Epoch: 123, Loss: 0.0050, Train: 100.00%, Valid: 72.20% Test: 72.60%
Run: 01, Epoch: 124, Loss: 0.0015, Train: 100.00%, Valid: 72.20%

Run: 02, Epoch: 19, Loss: 0.0620, Train: 71.67%, Valid: 64.60% Test: 66.20%
Run: 02, Epoch: 20, Loss: 0.0632, Train: 71.67%, Valid: 65.00% Test: 66.70%
Run: 02, Epoch: 21, Loss: 0.0278, Train: 73.33%, Valid: 65.20% Test: 67.40%
Run: 02, Epoch: 22, Loss: 0.0289, Train: 78.33%, Valid: 66.00% Test: 67.60%
Run: 02, Epoch: 23, Loss: 0.0254, Train: 80.00%, Valid: 66.40% Test: 68.90%
Run: 02, Epoch: 24, Loss: 0.0618, Train: 83.33%, Valid: 67.00% Test: 69.60%
Run: 02, Epoch: 25, Loss: 0.0377, Train: 83.33%, Valid: 67.80% Test: 70.80%
Run: 02, Epoch: 26, Loss: 0.0409, Train: 85.00%, Valid: 69.00% Test: 71.20%
Run: 02, Epoch: 27, Loss: 0.0296, Train: 91.67%, Valid: 69.20% Test: 71.80%
Run: 02, Epoch: 28, Loss: 0.0429, Train: 95.00%, Valid: 70.40% Test: 72.20%
Run: 02, Epoch: 29, Loss: 0.0317, Train: 96.67%, Valid: 71.60% Test: 73.10%
Run: 02, Epoch: 30, Loss: 0.0252, Train: 96.67%, Valid: 72.40% Test: 73.90%
Run: 02, Epoch: 31, Loss: 0.0398, Train: 96.67%, Valid: 72.60% Test: 74.60%
Run: 02, Epo

Run: 02, Epoch: 130, Loss: 0.0096, Train: 100.00%, Valid: 74.00% Test: 72.30%
Run: 02, Epoch: 131, Loss: 0.0041, Train: 100.00%, Valid: 74.00% Test: 72.50%
Run: 02, Epoch: 132, Loss: 0.0048, Train: 100.00%, Valid: 74.00% Test: 72.70%
Run: 02, Epoch: 133, Loss: 0.0009, Train: 100.00%, Valid: 74.00% Test: 72.50%
Run: 02, Epoch: 134, Loss: 0.0018, Train: 100.00%, Valid: 74.00% Test: 72.50%
Run: 02, Epoch: 135, Loss: 0.0104, Train: 100.00%, Valid: 74.00% Test: 72.80%
Run: 02, Epoch: 136, Loss: 0.0012, Train: 100.00%, Valid: 73.80% Test: 72.80%
Run: 02, Epoch: 137, Loss: 0.0023, Train: 100.00%, Valid: 73.80% Test: 72.80%
Run: 02, Epoch: 138, Loss: 0.0005, Train: 100.00%, Valid: 73.80% Test: 72.70%
Run: 02, Epoch: 139, Loss: 0.0038, Train: 100.00%, Valid: 73.80% Test: 72.90%
Run: 02, Epoch: 140, Loss: 0.0081, Train: 100.00%, Valid: 73.80% Test: 72.90%
Run: 02, Epoch: 141, Loss: 0.0048, Train: 100.00%, Valid: 73.80% Test: 72.90%
Run: 02, Epoch: 142, Loss: 0.0053, Train: 100.00%, Valid: 73.60%

Run: 03, Epoch: 42, Loss: 0.0033, Train: 100.00%, Valid: 75.80% Test: 74.90%
Run: 03, Epoch: 43, Loss: 0.0018, Train: 100.00%, Valid: 76.20% Test: 74.80%
Run: 03, Epoch: 44, Loss: 0.0162, Train: 100.00%, Valid: 76.00% Test: 74.80%
Run: 03, Epoch: 45, Loss: 0.0064, Train: 100.00%, Valid: 75.40% Test: 74.70%
Run: 03, Epoch: 46, Loss: 0.0145, Train: 100.00%, Valid: 75.60% Test: 74.50%
Run: 03, Epoch: 47, Loss: 0.0059, Train: 100.00%, Valid: 75.60% Test: 74.30%
Run: 03, Epoch: 48, Loss: 0.0100, Train: 100.00%, Valid: 75.40% Test: 74.10%
Run: 03, Epoch: 49, Loss: 0.0045, Train: 100.00%, Valid: 75.60% Test: 74.10%
Run: 03, Epoch: 50, Loss: 0.0038, Train: 100.00%, Valid: 75.40% Test: 74.10%
Run: 03, Epoch: 51, Loss: 0.0190, Train: 100.00%, Valid: 75.20% Test: 74.10%
Run: 03, Epoch: 52, Loss: 0.0033, Train: 100.00%, Valid: 75.00% Test: 74.20%
Run: 03, Epoch: 53, Loss: 0.0090, Train: 100.00%, Valid: 74.60% Test: 74.40%
Run: 03, Epoch: 54, Loss: 0.0359, Train: 100.00%, Valid: 74.20% Test: 74.10%

...

Run: 08, Epoch: 47, Loss: 0.0015, Train: 100.00%, Valid: 74.40% Test: 73.90%
Run: 08, Epoch: 48, Loss: 0.0075, Train: 100.00%, Valid: 74.40% Test: 73.70%
Run: 08, Epoch: 49, Loss: 0.0025, Train: 100.00%, Valid: 74.20% Test: 73.70%
Run: 08, Epoch: 50, Loss: 0.0043, Train: 100.00%, Valid: 74.40% Test: 73.80%
Run: 08, Epoch: 51, Loss: 0.0150, Train: 100.00%, Valid: 74.60% Test: 73.80%
Run: 08, Epoch: 52, Loss: 0.0074, Train: 100.00%, Valid: 74.60% Test: 73.60%
Run: 08, Epoch: 53, Loss: 0.0040, Train: 100.00%, Valid: 74.00% Test: 73.70%
Run: 08, Epoch: 54, Loss: 0.0139, Train: 100.00%, Valid: 74.00% Test: 73.60%
Run: 08, Epoch: 55, Loss: 0.0082, Train: 100.00%, Valid: 73.60% Test: 73.50%
Run: 08, Epoch: 56, Loss: 0.0029, Train: 100.00%, Valid: 73.60% Test: 73.30%
Run: 08, Epoch: 57, Loss: 0.0289, Train: 100.00%, Valid: 73.80% Test: 73.20%
Run: 08, Epoch: 58, Loss: 0.0020, Train: 100.00%, Valid: 74.00% Test: 73.40%
Run: 08, Epoch: 59, Loss: 0.0051, Train: 100.00%, Valid: 74.00% Test: 73.20%

Run: 08, Epoch: 163, Loss: 0.0028, Train: 100.00%, Valid: 71.60% Test: 70.80%
Run: 08, Epoch: 164, Loss: 0.0012, Train: 100.00%, Valid: 71.60% Test: 70.70%
Run: 08, Epoch: 165, Loss: 0.0015, Train: 100.00%, Valid: 71.60% Test: 70.80%
Run: 08, Epoch: 166, Loss: 0.0067, Train: 100.00%, Valid: 71.60% Test: 70.90%
Run: 08, Epoch: 167, Loss: 0.0054, Train: 100.00%, Valid: 71.40% Test: 71.20%
Run: 08, Epoch: 168, Loss: 0.0001, Train: 100.00%, Valid: 71.60% Test: 71.20%
Run: 08, Epoch: 169, Loss: 0.0071, Train: 100.00%, Valid: 71.40% Test: 71.10%
Run: 08, Epoch: 170, Loss: 0.0063, Train: 100.00%, Valid: 71.40% Test: 70.90%
Run: 08, Epoch: 171, Loss: 0.0036, Train: 100.00%, Valid: 71.20% Test: 70.80%
Run: 08, Epoch: 172, Loss: 0.0003, Train: 100.00%, Valid: 71.20% Test: 71.10%
Run: 08, Epoch: 173, Loss: 0.0015, Train: 100.00%, Valid: 71.40% Test: 71.00%
Run: 08, Epoch: 174, Loss: 0.0004, Train: 100.00%, Valid: 71.40% Test: 71.00%
Run: 08, Epoch: 175, Loss: 0.0044, Train: 100.00%, Valid: 71.20%

Run: 09, Epoch: 76, Loss: 0.0019, Train: 100.00%, Valid: 73.80% Test: 74.40%
Run: 09, Epoch: 77, Loss: 0.0046, Train: 100.00%, Valid: 73.80% Test: 74.40%
Run: 09, Epoch: 78, Loss: 0.0116, Train: 100.00%, Valid: 73.80% Test: 74.30%
Run: 09, Epoch: 79, Loss: 0.0089, Train: 100.00%, Valid: 73.80% Test: 74.50%
Run: 09, Epoch: 80, Loss: 0.0016, Train: 100.00%, Valid: 73.80% Test: 74.50%
Run: 09, Epoch: 81, Loss: 0.0007, Train: 100.00%, Valid: 73.60% Test: 74.70%
Run: 09, Epoch: 82, Loss: 0.0161, Train: 100.00%, Valid: 73.20% Test: 74.70%
Run: 09, Epoch: 83, Loss: 0.0009, Train: 100.00%, Valid: 73.40% Test: 74.60%
Run: 09, Epoch: 84, Loss: 0.0049, Train: 100.00%, Valid: 73.20% Test: 74.60%
Run: 09, Epoch: 85, Loss: 0.0309, Train: 100.00%, Valid: 73.20% Test: 74.40%
Run: 09, Epoch: 86, Loss: 0.0023, Train: 100.00%, Valid: 72.80% Test: 74.30%
Run: 09, Epoch: 87, Loss: 0.0009, Train: 100.00%, Valid: 72.80% Test: 74.10%
Run: 09, Epoch: 88, Loss: 0.0027, Train: 100.00%, Valid: 72.80% Test: 74.00%

Run: 09, Epoch: 188, Loss: 0.0036, Train: 100.00%, Valid: 72.20% Test: 71.80%
Run: 09, Epoch: 189, Loss: 0.0021, Train: 100.00%, Valid: 72.20% Test: 71.70%
Run: 09, Epoch: 190, Loss: 0.0165, Train: 100.00%, Valid: 72.20% Test: 71.70%
Run: 09, Epoch: 191, Loss: 0.0012, Train: 100.00%, Valid: 72.20% Test: 71.70%
Run: 09, Epoch: 192, Loss: 0.0156, Train: 100.00%, Valid: 72.20% Test: 71.90%
Run: 09, Epoch: 193, Loss: 0.0155, Train: 100.00%, Valid: 72.40% Test: 72.30%
Run: 09, Epoch: 194, Loss: 0.0021, Train: 100.00%, Valid: 72.60% Test: 72.30%
Run: 09, Epoch: 195, Loss: 0.0009, Train: 100.00%, Valid: 72.80% Test: 72.50%
Run: 09, Epoch: 196, Loss: 0.0018, Train: 100.00%, Valid: 72.80% Test: 72.70%
Run: 09, Epoch: 197, Loss: 0.0007, Train: 100.00%, Valid: 72.60% Test: 72.70%
Run: 09, Epoch: 198, Loss: 0.0002, Train: 100.00%, Valid: 72.80% Test: 72.80%
Run: 09, Epoch: 199, Loss: 0.0008, Train: 100.00%, Valid: 72.60% Test: 72.80%
Run: 09, Epoch: 200, Loss: 0.0009, Train: 100.00%, Valid: 72.40%

Run: 10, Epoch: 99, Loss: 0.0021, Train: 100.00%, Valid: 70.60% Test: 70.90%
Run: 10, Epoch: 100, Loss: 0.0026, Train: 100.00%, Valid: 70.80% Test: 70.90%
Run: 10, Epoch: 101, Loss: 0.0201, Train: 100.00%, Valid: 70.80% Test: 71.20%
Run: 10, Epoch: 102, Loss: 0.0089, Train: 100.00%, Valid: 70.60% Test: 71.30%
Run: 10, Epoch: 103, Loss: 0.0010, Train: 100.00%, Valid: 70.60% Test: 71.50%
Run: 10, Epoch: 104, Loss: 0.0057, Train: 100.00%, Valid: 70.00% Test: 71.50%
Run: 10, Epoch: 105, Loss: 0.0116, Train: 100.00%, Valid: 70.20% Test: 71.50%
Run: 10, Epoch: 106, Loss: 0.0057, Train: 100.00%, Valid: 70.20% Test: 71.30%
Run: 10, Epoch: 107, Loss: 0.0154, Train: 100.00%, Valid: 70.80% Test: 71.20%
Run: 10, Epoch: 108, Loss: 0.0017, Train: 100.00%, Valid: 71.00% Test: 71.60%
Run: 10, Epoch: 109, Loss: 0.0111, Train: 100.00%, Valid: 71.20% Test: 71.60%
Run: 10, Epoch: 110, Loss: 0.0023, Train: 100.00%, Valid: 71.20% Test: 71.40%
Run: 10, Epoch: 111, Loss: 0.0211, Train: 100.00%, Valid: 71.00% 

# Wise Embedding

In [12]:
dataset = Planetoid(root='/tmp/PubMed', name='PubMed',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[19717, 500], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], adj_t=[19717, 19717, nnz=88648])


In [13]:
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,class
0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.004999,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.016434,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.104636,0.0,0.0,0.0,0.035178,0.0,0.0,0.019555,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.008582,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.007356,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [14]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=data.y.numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}

In [15]:
import numpy as np
def Average(lst):
    # average function
    avg = np.average(lst)
    return(avg)
sel_basis = [[Average(list(df[i].to_numpy())) for i in range(len(df.columns))]for df in data_by_class.values()]

In [16]:
feature_names = [ii for ii in range(fe_len)]
Fec=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    for j in range(max(catagories)+1):
        vec.append(np.linalg.norm(np.array(f) - np.array(sel_basis[j])))
    f.clear()
    Fec.append(vec)

Processing file 19716 (100%)

In [14]:
print(Fec[0])

[0.2534605892190711, 0.2233912937204271, 0.23581046385004045]


In [17]:
CC_domain=torch.tensor(Fec).float()
data.x=CC_domain
print(data)

Data(x=[19717, 3], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], adj_t=[19717, 19717, nnz=88648])


# W-GCN

In [15]:
data.x=CC_domain
print(data)

Data(x=[19717, 3], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], adj_t=[19717, 19717, nnz=88648])


In [16]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GCN(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.5036, Train: 33.33%, Valid: 41.60% Test: 40.70%
Run: 01, Epoch: 02, Loss: 1.3778, Train: 33.33%, Valid: 41.60% Test: 41.60%
Run: 01, Epoch: 03, Loss: 1.1679, Train: 31.67%, Valid: 49.00% Test: 48.90%
Run: 01, Epoch: 04, Loss: 1.3058, Train: 33.33%, Valid: 39.00% Test: 42.60%
Run: 01, Epoch: 05, Loss: 1.1687, Train: 33.33%, Valid: 38.80% Test: 41.30%
Run: 01, Epoch: 06, Loss: 1.0682, Train: 33.33%, Valid: 38.80% Test: 41.30%
Run: 01, Epoch: 07, Loss: 0.9813, Train: 33.33%, Valid: 38.80% Test: 41.30%
Run: 01, Epoch: 08, Loss: 1.0364, Train: 33.33%, Valid: 38.80% Test: 41.30%
Run: 01, Epoch: 09, Loss: 0.9612, Train: 33.33%, Valid: 38.80% Test: 41.30%
Run: 01, Epoch: 10, Loss: 0.9135, Train: 33.33%, Valid: 38.80% Test: 41.30%
Run: 01, Epoch: 11, Loss: 0.9181, Train: 33.33%, Valid: 38.80% Test: 41.30%
Run: 01, Epoch: 12, Loss: 0.8821, Train: 33.33%, Valid: 38.80% Test: 41.30%
Run: 01, Epoch: 13, Loss: 0.7642, Train: 33.33%, Valid: 38.80% Test: 41.30%
Run: 01, Epo

Run: 01, Epoch: 124, Loss: 0.3486, Train: 63.33%, Valid: 53.80% Test: 51.60%
Run: 01, Epoch: 125, Loss: 0.4844, Train: 78.33%, Valid: 68.80% Test: 64.80%
Run: 01, Epoch: 126, Loss: 0.3702, Train: 76.67%, Valid: 71.00% Test: 69.00%
Run: 01, Epoch: 127, Loss: 0.3895, Train: 66.67%, Valid: 63.00% Test: 60.70%
Run: 01, Epoch: 128, Loss: 0.3898, Train: 56.67%, Valid: 54.80% Test: 51.80%
Run: 01, Epoch: 129, Loss: 0.3928, Train: 46.67%, Valid: 42.20% Test: 40.40%
Run: 01, Epoch: 130, Loss: 0.3420, Train: 41.67%, Valid: 31.60% Test: 30.10%
Run: 01, Epoch: 131, Loss: 0.3415, Train: 33.33%, Valid: 27.40% Test: 27.60%
Run: 01, Epoch: 132, Loss: 0.3593, Train: 36.67%, Valid: 29.80% Test: 28.90%
Run: 01, Epoch: 133, Loss: 0.3515, Train: 55.00%, Valid: 42.60% Test: 40.30%
Run: 01, Epoch: 134, Loss: 0.4181, Train: 66.67%, Valid: 52.80% Test: 50.40%
Run: 01, Epoch: 135, Loss: 0.3769, Train: 65.00%, Valid: 55.20% Test: 52.40%
Run: 01, Epoch: 136, Loss: 0.3631, Train: 65.00%, Valid: 54.80% Test: 52.20%

Run: 02, Epoch: 39, Loss: 0.4988, Train: 36.67%, Valid: 43.00% Test: 43.10%
Run: 02, Epoch: 40, Loss: 0.5947, Train: 33.33%, Valid: 41.80% Test: 41.60%
Run: 02, Epoch: 41, Loss: 0.5070, Train: 33.33%, Valid: 41.60% Test: 40.80%
Run: 02, Epoch: 42, Loss: 0.4547, Train: 33.33%, Valid: 41.60% Test: 40.70%
Run: 02, Epoch: 43, Loss: 0.5623, Train: 33.33%, Valid: 41.60% Test: 40.80%
Run: 02, Epoch: 44, Loss: 0.4660, Train: 33.33%, Valid: 41.60% Test: 40.80%
Run: 02, Epoch: 45, Loss: 0.4677, Train: 33.33%, Valid: 41.80% Test: 41.60%
Run: 02, Epoch: 46, Loss: 0.4563, Train: 36.67%, Valid: 41.80% Test: 42.30%
Run: 02, Epoch: 47, Loss: 0.4900, Train: 36.67%, Valid: 43.20% Test: 43.10%
Run: 02, Epoch: 48, Loss: 0.4909, Train: 45.00%, Valid: 46.60% Test: 45.20%
Run: 02, Epoch: 49, Loss: 0.5022, Train: 55.00%, Valid: 49.00% Test: 47.60%
Run: 02, Epoch: 50, Loss: 0.4929, Train: 58.33%, Valid: 52.00% Test: 49.60%
Run: 02, Epoch: 51, Loss: 0.5520, Train: 58.33%, Valid: 53.60% Test: 50.40%
Run: 02, Epo

Run: 02, Epoch: 160, Loss: 0.4374, Train: 40.00%, Valid: 45.00% Test: 48.30%
Run: 02, Epoch: 161, Loss: 0.4039, Train: 60.00%, Valid: 51.80% Test: 55.40%
Run: 02, Epoch: 162, Loss: 0.3658, Train: 63.33%, Valid: 58.60% Test: 59.60%
Run: 02, Epoch: 163, Loss: 0.3712, Train: 63.33%, Valid: 55.20% Test: 56.50%
Run: 02, Epoch: 164, Loss: 0.3822, Train: 33.33%, Valid: 42.20% Test: 43.10%
Run: 02, Epoch: 165, Loss: 0.4121, Train: 33.33%, Valid: 41.80% Test: 41.00%
Run: 02, Epoch: 166, Loss: 0.4686, Train: 35.00%, Valid: 42.00% Test: 41.50%
Run: 02, Epoch: 167, Loss: 0.4905, Train: 50.00%, Valid: 48.00% Test: 47.00%
Run: 02, Epoch: 168, Loss: 0.3842, Train: 75.00%, Valid: 60.20% Test: 58.60%
Run: 02, Epoch: 169, Loss: 0.4291, Train: 75.00%, Valid: 60.60% Test: 58.00%
Run: 02, Epoch: 170, Loss: 0.3618, Train: 68.33%, Valid: 51.20% Test: 50.60%
Run: 02, Epoch: 171, Loss: 0.3620, Train: 63.33%, Valid: 47.40% Test: 46.80%
Run: 02, Epoch: 172, Loss: 0.3555, Train: 63.33%, Valid: 52.80% Test: 50.10%

Run: 03, Epoch: 86, Loss: 0.3661, Train: 43.33%, Valid: 42.00% Test: 43.60%
Run: 03, Epoch: 87, Loss: 0.3622, Train: 33.33%, Valid: 39.20% Test: 41.60%
Run: 03, Epoch: 88, Loss: 0.3789, Train: 33.33%, Valid: 38.80% Test: 41.40%
Run: 03, Epoch: 89, Loss: 0.3456, Train: 33.33%, Valid: 38.80% Test: 41.30%
Run: 03, Epoch: 90, Loss: 0.3518, Train: 33.33%, Valid: 38.80% Test: 41.30%
Run: 03, Epoch: 91, Loss: 0.3232, Train: 33.33%, Valid: 38.80% Test: 41.30%
Run: 03, Epoch: 92, Loss: 0.4285, Train: 33.33%, Valid: 38.80% Test: 41.30%
Run: 03, Epoch: 93, Loss: 0.4597, Train: 33.33%, Valid: 38.80% Test: 41.40%
Run: 03, Epoch: 94, Loss: 0.3842, Train: 33.33%, Valid: 38.80% Test: 41.40%
Run: 03, Epoch: 95, Loss: 0.4138, Train: 33.33%, Valid: 38.80% Test: 41.40%
Run: 03, Epoch: 96, Loss: 0.2979, Train: 33.33%, Valid: 38.80% Test: 41.50%
Run: 03, Epoch: 97, Loss: 0.4551, Train: 33.33%, Valid: 38.80% Test: 41.40%
Run: 03, Epoch: 98, Loss: 0.4178, Train: 33.33%, Valid: 39.20% Test: 41.70%
Run: 03, Epo

...

Run: 08, Epoch: 29, Loss: 0.6965, Train: 48.33%, Valid: 37.40% Test: 38.30%
Run: 08, Epoch: 30, Loss: 0.7026, Train: 48.33%, Valid: 41.80% Test: 42.00%
Run: 08, Epoch: 31, Loss: 0.6903, Train: 51.67%, Valid: 44.80% Test: 43.40%
Run: 08, Epoch: 32, Loss: 0.6821, Train: 48.33%, Valid: 45.80% Test: 45.30%
Run: 08, Epoch: 33, Loss: 0.6687, Train: 48.33%, Valid: 43.40% Test: 44.80%
Run: 08, Epoch: 34, Loss: 0.6541, Train: 48.33%, Valid: 41.80% Test: 43.40%
Run: 08, Epoch: 35, Loss: 0.7365, Train: 45.00%, Valid: 42.00% Test: 44.20%
Run: 08, Epoch: 36, Loss: 0.6228, Train: 45.00%, Valid: 41.80% Test: 43.80%
Run: 08, Epoch: 37, Loss: 0.6621, Train: 45.00%, Valid: 41.60% Test: 43.70%
Run: 08, Epoch: 38, Loss: 0.5928, Train: 45.00%, Valid: 42.40% Test: 43.70%
Run: 08, Epoch: 39, Loss: 0.5836, Train: 41.67%, Valid: 43.00% Test: 43.20%
Run: 08, Epoch: 40, Loss: 0.5503, Train: 40.00%, Valid: 41.60% Test: 43.60%
Run: 08, Epoch: 41, Loss: 0.5979, Train: 40.00%, Valid: 41.40% Test: 43.30%
Run: 08, Epo

Run: 08, Epoch: 145, Loss: 0.3837, Train: 58.33%, Valid: 52.80% Test: 51.00%
Run: 08, Epoch: 146, Loss: 0.3477, Train: 65.00%, Valid: 56.60% Test: 53.80%
Run: 08, Epoch: 147, Loss: 0.3502, Train: 63.33%, Valid: 53.20% Test: 50.60%
Run: 08, Epoch: 148, Loss: 0.3137, Train: 56.67%, Valid: 48.60% Test: 45.30%
Run: 08, Epoch: 149, Loss: 0.3734, Train: 56.67%, Valid: 47.20% Test: 45.30%
Run: 08, Epoch: 150, Loss: 0.3636, Train: 58.33%, Valid: 48.00% Test: 45.90%
Run: 08, Epoch: 151, Loss: 0.3182, Train: 61.67%, Valid: 52.40% Test: 50.40%
Run: 08, Epoch: 152, Loss: 0.3364, Train: 61.67%, Valid: 50.40% Test: 48.70%
Run: 08, Epoch: 153, Loss: 0.4220, Train: 46.67%, Valid: 35.20% Test: 33.90%
Run: 08, Epoch: 154, Loss: 0.4818, Train: 33.33%, Valid: 21.40% Test: 19.20%
Run: 08, Epoch: 155, Loss: 0.3920, Train: 33.33%, Valid: 19.60% Test: 18.00%
Run: 08, Epoch: 156, Loss: 0.3290, Train: 36.67%, Valid: 23.00% Test: 22.70%
Run: 08, Epoch: 157, Loss: 0.4165, Train: 51.67%, Valid: 45.40% Test: 44.10%

Run: 09, Epoch: 61, Loss: 0.5020, Train: 35.00%, Valid: 42.20% Test: 40.90%
Run: 09, Epoch: 62, Loss: 0.4768, Train: 33.33%, Valid: 42.00% Test: 40.90%
Run: 09, Epoch: 63, Loss: 0.4348, Train: 40.00%, Valid: 42.60% Test: 41.20%
Run: 09, Epoch: 64, Loss: 0.4217, Train: 48.33%, Valid: 45.40% Test: 43.60%
Run: 09, Epoch: 65, Loss: 0.4273, Train: 60.00%, Valid: 51.40% Test: 50.90%
Run: 09, Epoch: 66, Loss: 0.4429, Train: 76.67%, Valid: 62.00% Test: 62.20%
Run: 09, Epoch: 67, Loss: 0.4170, Train: 71.67%, Valid: 64.40% Test: 64.60%
Run: 09, Epoch: 68, Loss: 0.4827, Train: 63.33%, Valid: 65.80% Test: 65.60%
Run: 09, Epoch: 69, Loss: 0.3847, Train: 50.00%, Valid: 51.80% Test: 55.00%
Run: 09, Epoch: 70, Loss: 0.4727, Train: 33.33%, Valid: 42.40% Test: 43.70%
Run: 09, Epoch: 71, Loss: 0.3832, Train: 33.33%, Valid: 39.20% Test: 41.60%
Run: 09, Epoch: 72, Loss: 0.4121, Train: 33.33%, Valid: 38.80% Test: 41.50%
Run: 09, Epoch: 73, Loss: 0.4534, Train: 35.00%, Valid: 39.80% Test: 42.00%
Run: 09, Epo

Run: 09, Epoch: 177, Loss: 0.3404, Train: 38.33%, Valid: 40.60% Test: 42.30%
Run: 09, Epoch: 178, Loss: 0.3248, Train: 36.67%, Valid: 41.20% Test: 43.20%
Run: 09, Epoch: 179, Loss: 0.4918, Train: 63.33%, Valid: 59.20% Test: 61.70%
Run: 09, Epoch: 180, Loss: 0.3244, Train: 65.00%, Valid: 56.60% Test: 58.50%
Run: 09, Epoch: 181, Loss: 0.3052, Train: 50.00%, Valid: 50.60% Test: 48.60%
Run: 09, Epoch: 182, Loss: 0.4338, Train: 48.33%, Valid: 47.80% Test: 45.80%
Run: 09, Epoch: 183, Loss: 0.3013, Train: 48.33%, Valid: 47.40% Test: 45.00%
Run: 09, Epoch: 184, Loss: 0.3257, Train: 48.33%, Valid: 47.20% Test: 44.80%
Run: 09, Epoch: 185, Loss: 0.3041, Train: 48.33%, Valid: 46.60% Test: 44.20%
Run: 09, Epoch: 186, Loss: 0.3826, Train: 45.00%, Valid: 45.00% Test: 42.70%
Run: 09, Epoch: 187, Loss: 0.3706, Train: 38.33%, Valid: 42.80% Test: 41.40%
Run: 09, Epoch: 188, Loss: 0.2772, Train: 33.33%, Valid: 42.00% Test: 40.90%
Run: 09, Epoch: 189, Loss: 0.3202, Train: 33.33%, Valid: 42.00% Test: 40.90%

Run: 10, Epoch: 91, Loss: 0.3804, Train: 66.67%, Valid: 64.20% Test: 63.50%
Run: 10, Epoch: 92, Loss: 0.3509, Train: 75.00%, Valid: 70.60% Test: 71.40%
Run: 10, Epoch: 93, Loss: 0.4663, Train: 70.00%, Valid: 68.00% Test: 66.10%
Run: 10, Epoch: 94, Loss: 0.3906, Train: 61.67%, Valid: 55.80% Test: 53.90%
Run: 10, Epoch: 95, Loss: 0.3290, Train: 63.33%, Valid: 55.40% Test: 53.70%
Run: 10, Epoch: 96, Loss: 0.3243, Train: 63.33%, Valid: 55.60% Test: 53.90%
Run: 10, Epoch: 97, Loss: 0.4286, Train: 61.67%, Valid: 54.20% Test: 53.10%
Run: 10, Epoch: 98, Loss: 0.3296, Train: 60.00%, Valid: 52.80% Test: 51.80%
Run: 10, Epoch: 99, Loss: 0.4566, Train: 60.00%, Valid: 52.40% Test: 51.50%
Run: 10, Epoch: 100, Loss: 0.4645, Train: 61.67%, Valid: 52.80% Test: 52.30%
Run: 10, Epoch: 101, Loss: 0.4332, Train: 63.33%, Valid: 56.00% Test: 53.80%
Run: 10, Epoch: 102, Loss: 0.4075, Train: 60.00%, Valid: 53.40% Test: 51.80%
Run: 10, Epoch: 103, Loss: 0.3465, Train: 48.33%, Valid: 46.80% Test: 46.20%
Run: 10,

# Topological Encoding 

In [ ]:
dataset = Planetoid(root='/tmp/PubMed', name='PubMed')
data = dataset[0]
print(data)

In [ ]:
print(data.edge_index.numpy())

In [ ]:
Edge_idx=data.edge_index.numpy()
Node=range(Number_nodes)
Edgelist=[]
for i in range(len(Edge_idx[1])):
    Edgelist.append((Edge_idx[0][i],Edge_idx[1][i]))
#print(Edgelist)

In [ ]:
# a "plain" graph is undirected
G = nx.DiGraph()

# give each a node a 'name', which is a letter in this case.
#G.add_node('a')

# the add_nodes_from method allows adding nodes from a sequence, in this case a list
#nodes_to_add = ['b', 'c', 'd']
G.add_nodes_from(Node)

# add edge from 'a' to 'b'
# since this graph is undirected, the order doesn't matter here
#G.add_edge('a', 'b')

# just like add_nodes_from, we can add edges from a sequence
# edges should be specified as 2-tuples
#edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(Edgelist)


In [ ]:
print(G.number_of_edges())

In [ ]:
def Topological_Feature_subLevel(adj,filtration_fun, Filtration):
        betti_0=[]
        betti_1=[]
        for p in range(len(Filtration)):
            n_active = np.where(np.array(filtration_fun) <= Filtration[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
                betti_1.append(0)
            else:
                b=adj[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
                betti_1.append(x[1])
            n_active.clear()
        return betti_0,betti_1

In [ ]:
def Degree_list(Graph):
    degree_list = [Graph.degree(node) for node in Graph.nodes]
    return np.array(degree_list)

In [ ]:
degree_list=Degree_list(G)
unique_list=np.unique(degree_list)
for d in unique_list:
    count=0
    for i in range(len(degree_list)):
        if degree_list[i]==d:
            count=count+1
    print(int(d)," | ",count,'\n')

In [ ]:
import pyflagser
Node_fil=[0,2,4,6,8,10,12,14,16,18,20,22,24,30,34]
topo_betti_0=[]
topo_betti_1=[]
Node_Edge=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    subgraph=ego_graph(G, i, radius=2, center=False, undirected=True, distance=None)
    filt=Degree_list(subgraph)
    A_sub = nx.to_numpy_array(subgraph)# adjacency matrix of subgraph
    fe=Topological_Feature_subLevel(A_sub,filt,Node_fil)
    topo_betti_0.append(fe[0])
    topo_betti_1.append(fe[1])
    Node_Edge.append([subgraph.number_of_nodes(),subgraph.number_of_edges()])
    #topo_with_NE.app

In [ ]:
print( Node_Edge)

In [2]:
import pandas as pd
data = pd.read_csv('Feature_Pubmed.csv')
data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,11.1,12.1,13.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1
0,0,72,87,94,94,96,96,96,91,91,...,0,0,0,0,0,1,4,4,7,11
1,1,20,25,28,25,25,21,21,21,21,...,1,1,1,1,1,5,5,5,5,5
2,2,42,43,44,44,44,44,44,44,44,...,0,0,0,0,0,0,0,0,0,0
3,3,10,14,15,12,12,12,12,12,12,...,0,0,0,0,0,0,0,0,0,0
4,4,6,29,30,31,30,30,30,30,30,...,0,0,0,0,0,0,0,1,0,0


In [3]:
Data1=data.drop(['Unnamed: 0'], axis=1)
Data1.head()

,0,1,2,3,4,5,6,7,8,9,...,11.1,12.1,13.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1
0,72,87,94,94,96,96,96,91,91,91,...,0,0,0,0,0,1,4,4,7,11
1,20,25,28,25,25,21,21,21,21,21,...,1,1,1,1,1,5,5,5,5,5
2,42,43,44,44,44,44,44,44,44,44,...,0,0,0,0,0,0,0,0,0,0
3,10,14,15,12,12,12,12,12,12,12,...,0,0,0,0,0,0,0,0,0,0
4,6,29,30,31,30,30,30,30,30,30,...,0,0,0,0,0,0,0,1,0,0


In [6]:
topo_fe=torch.tensor(Data1.values).float()

In [9]:
dataset = Planetoid(root='/tmp/PubMed', name='PubMed',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[19717, 500], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], adj_t=[19717, 19717, nnz=88648])


In [18]:
data.x=CC_domain
data.topo=topo_fe.float()
print(data)

Data(x=[19717, 3], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], adj_t=[19717, 19717, nnz=88648], topo=[19717, 42])


In [70]:
print(len(topo_fe[0]))

42


In [15]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        #return x.log_softmax(dim=-1)
        return x

class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        #return torch.log_softmax(x, dim=-1)
        return x
    
class MLP2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP2, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp2(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)
    

def train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2):
    model.train()
    mlp_model.train()
    mlp_2.train()
    optimizer.zero_grad()
    optimizer_mlp.zero_grad()
    optimizer_mlp2.zero_grad()
    gcn_embedding = model(data.x, data.adj_t)[train_idx]
    #print(gcn_embedding)
    mlp_embedding = mlp_model(data.topo[train_idx])
    #print(mlp_embedding)
    combined_embedding = torch.cat((gcn_embedding, mlp_embedding), dim=1)
    #print(combined_embedding)
    mlp_emb = mlp_2(combined_embedding)
    #print(mlp_emb)
    loss = F.nll_loss(mlp_emb, data.y.squeeze()[train_idx])
    #loss = F.nll_loss(combined_embedding, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer_mlp2.step()
    optimizer.step()
    optimizer_mlp.step()
    

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total



@torch.no_grad()
def test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx):
    model.eval()
    mlp_model.eval()
    mlp_2.eval()

    gcn_out = model(data.x, data.adj_t)
    #print(gcn_out[0])
    mlp_out=mlp_model(data.topo)
    #print(mlp_out)
    #out=torch.cat((gcn_out,mlp_out),dim=1)
    Com=torch.cat((gcn_out,mlp_out),dim=1)
    out=mlp_2(Com)
    y_pred = out.argmax(dim=-1, keepdim=True)
    #print(y_pred[0])
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [16]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GCN(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 50, dataset.num_classes,3, 0.3)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.01)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
               # print(f'Run: {run + 1:02d}, 'f'Epoch: {epoch:02d}, 'f'Loss: {loss:.4f}, 'f'Train: {100 * train_acc:.2f}%, '
                #      f'Valid: {100 * valid_acc:.2f}% '
                 #     f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

/Users/joshem/anaconda3/envs/tensorflow/lib/python3.10/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Run 01:
Highest Train: 100.00
Highest Valid: 76.60
  Final Train: 100.00
   Final Test: 76.10
Run 02:
Highest Train: 100.00
Highest Valid: 77.00
  Final Train: 100.00
   Final Test: 75.70
Run 03:
Highest Train: 100.00
Highest Valid: 76.00
  Final Train: 100.00
   Final Test: 76.40
Run 04:
Highest Train: 100.00
Highest Valid: 77.60
  Final Train: 100.00
   Final Test: 76.80
Run 05:
Highest Train: 100.00
Highest Valid: 78.00
  Final Train: 100.00
   Final Test: 76.70
Run 06:
Highest Train: 100.00
Highest Valid: 76.20
  Final Train: 100.00
   Final Test: 76.00
Run 07:
Highest Train: 100.00
Highest Valid: 75.60
  Final Train: 100.00
   Final Test: 76.20
Run 08:
Highest Train: 100.00
Highest Valid: 77.40
  Final Train: 100.00
   Final Test: 76.10
Run 09:
Highest Train: 100.00
Highest Valid: 77.20
  Final Train: 100.00
   Final Test: 73.60
Run 10:
Highest Train: 100.00
Highest Valid: 77.40
  Final Train: 100.00
   Final Test: 75.70
All runs:
Highest Train: 100.00 ± 0.00
Highest Valid: 76.90 

# TOPO-GCN

In [19]:
dataset = Planetoid(root='/tmp/PubMed', name='PubMed',transform=T.ToSparseTensor())
data = dataset[0]
data.topo=topo_fe.float()
print(data)

Data(x=[19717, 500], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717], adj_t=[19717, 19717, nnz=88648], topo=[19717, 42])


In [ ]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GCN(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 50, dataset.num_classes,3, 0.3)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.01)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
               # print(f'Run: {run + 1:02d}, 'f'Epoch: {epoch:02d}, 'f'Loss: {loss:.4f}, 'f'Train: {100 * train_acc:.2f}%, '
                #      f'Valid: {100 * valid_acc:.2f}% '
                 #     f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()